# Build a simple ML pipeline for image classification

## Introduction
This tutorial shows how to train a simple deep neural network using the [Fashion MNIST dataset and Keras on Azure Machine Learning. Fashion-MNIST is a dataset of Zalando's article imagesâ€”consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.


## Set up your development environment

All the setup for your development work can be accomplished in a Python notebook.  Setup includes:

### Import packages

Import Python packages you need in this session. Also display the Azure Machine Learning SDK version.

In [1]:
import os
import azureml.core
from azureml.core import (
    Workspace,
    Dataset,
    Datastore,
    ComputeTarget,
    Experiment,
    ScriptRunConfig,
)
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.38.0


### Connect to workspace

Create a workspace object from the existing workspace. `Workspace.from_config()` reads the file **config.json** and loads the details into an object named `workspace`.

In [3]:
# load workspace
workspace = Workspace.from_config()
print(
    "Workspace name: " + workspace.name,
    "Azure region: " + workspace.location,
    "Subscription id: " + workspace.subscription_id,
    "Resource group: " + workspace.resource_group,
    sep="\n",
)

Workspace name: aml-workspace
Azure region: westeurope
Subscription id: b17f1c19-34a2-47b8-a207-40ea477828fc
Resource group: aml-resource-group


### Create experiment and a directory

Create an experiment to track the runs in your workspace and a directory to deliver the necessary code from your computer to the remote resource.

In [7]:
# create an ML experiment
exp = Experiment(workspace=workspace, name="keras-mnist-fashion")

# create a directory
script_folder = "./keras-mnist-fashion"
os.makedirs(script_folder, exist_ok=True)

### Create or Attach existing compute resource

**Creation of compute takes approximately 5 minutes.** If the AmlCompute with that name is already in your workspace the code will skip the creation process.

In [9]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpu-cluster" #"gpu-cluster"

found = False
# Check if this compute target already exists in the workspace.
cts = workspace.compute_targets
if cluster_name in cts and cts[cluster_name].type == "AmlCompute":
    found = True
    print("Found existing compute target.")
    compute_target = cts[cluster_name]
if not found:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_D12_v2",  # for GPU, use "STANDARD_NC6"
        vm_priority = 'lowpriority', # optional
        max_nodes=4,
    )
    
    # Create the cluster.
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    
    # Can poll for a minimum number of nodes and for a specific timeout.
    # If no min_node_count is provided, it will use the scale settings for the cluster.
    compute_target.wait_for_completion(
        show_output=True, min_node_count=None, timeout_in_minutes=10
    )
# For a more detailed view of current AmlCompute status, use get_status().print(compute_target.get_status().serialize())

Creating a new compute target...
InProgress....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Create the Fashion MNIST dataset

By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred.

In [13]:
data_urls = ["https://data4mldemo6150520719.blob.core.windows.net/demo/mnist-fashion"]
fashion_ds = Dataset.File.from_files(data_urls)

# list the files referenced by fashion_ds
fashion_ds.to_path()

['/mnist-fashion/t10k-images-idx3-ubyte',
 '/mnist-fashion/t10k-labels-idx1-ubyte',
 '/mnist-fashion/train-images-idx3-ubyte',
 '/mnist-fashion/train-labels-idx1-ubyte']

## Build 2-step ML pipeline

### Step 1: data preparation

In step one, we will load the image and labels from Fashion MNIST dataset into mnist_train.csv and mnist_test.csv

Intermediate data (or output of a step) is represented by a `OutputFileDatasetConfig` object. preprared_fashion_ds is produced as the output of step 1, and used as the input of step 2. `OutputFileDatasetConfig` introduces a data dependency between steps, and creates an implicit execution order in the pipeline. You can register a `OutputFileDatasetConfig` as a dataset and version the output data automatically.

In [16]:
from azureml.data import OutputFileDatasetConfig

# learn more about the output config
help(OutputFileDatasetConfig)

Help on class OutputFileDatasetConfig in module azureml.data.output_dataset_config:

class OutputFileDatasetConfig(OutputDatasetConfig, TransformationMixin)
 |  OutputFileDatasetConfig(name=None, destination=None, source=None, partition_format=None)
 |  
 |  Represent how to copy the output of a run and be promoted as a FileDataset.
 |  
 |  The OutputFileDatasetConfig allows you to specify how you want a particular local path on the compute target
 |  to be uploaded to the specified destination. If no arguments are passed to the constructor, we will
 |  automatically generate a name, a destination, and a local path.
 |  
 |  An example of not passing any arguments:
 |  
 |  .. code-block:: python
 |  
 |      workspace = Workspace.from_config()
 |      experiment = Experiment(workspace, 'output_example')
 |  
 |      output = OutputFileDatasetConfig()
 |  
 |      script_run_config = ScriptRunConfig('.', 'train.py', arguments=[output])
 |  
 |      run = experiment.submit(script_run_c

In [19]:
# write output to datastore under folder `outputdataset` and register it as a dataset after the experiment completes
# make sure the service principal in your datastore has blob data contributor role in order to write data back
datastore = workspace.get_default_datastore()
prepared_fashion_ds = OutputFileDatasetConfig(
    destination=(datastore, "outputdataset/{run-id}")
).register_on_complete(name="prepared_fashion_ds")

A **PythonScriptStep** is a basic, built-in step to run a Python Script on a compute target. It takes a script name and optionally other parameters like arguments for the script, compute target, inputs and outputs. If no compute target is specified, default compute target for the workspace is used. 

In [26]:
prep_step = PythonScriptStep(
    name="prepare step",
    script_name="prepare.py",
    # mount fashion_ds dataset to the compute_target
    arguments=[fashion_ds.as_named_input("fashion_ds").as_mount(), prepared_fashion_ds],
    source_directory=script_folder,
    compute_target=compute_target,
    allow_reuse=True,
)

The code in prepare.py takes two command-line arguments: the first is assigned to mounted_input_path and the second to mounted_output_path. If that subdirectory doesn't exist, the call to os.makedirs creates it. Then, the program converts the training and testing data and outputs the comma-separated files to the mounted_output_path.

### Step 2: train CNN with Keras

Next, construct a ScriptRunConfig to configure the training run that trains a CNN model using Keras. It takes a dataset as the input.

In [27]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- pip:
  - azureml-core
  - azureml-dataset-runtime
  - keras==2.4.3
  - tensorflow==2.4.3
  - numpy
  - scikit-learn
  - pandas
  - matplotlib

Writing conda_dependencies.yml


In [28]:
from azureml.core import Environment

keras_env = Environment.from_conda_specification(
    name="keras-env", file_path="./conda_dependencies.yml"
)

In [29]:
train_src = ScriptRunConfig(
    source_directory=script_folder,
    script="train.py",
    compute_target=compute_target,
    environment=keras_env,
)

Pass the run configuration details into the PythonScriptStep.

In [30]:
train_step = PythonScriptStep(
    name="train step",
    arguments=[
        prepared_fashion_ds.read_delimited_files().as_input(name="prepared_fashion_ds")
    ],
    source_directory=train_src.source_directory,
    script_name=train_src.script,
    runconfig=train_src.run_config,
)

### Build the pipeline
Once we have the steps (or steps collection), we can build the pipeline.

A pipeline is created with a list of steps and a workspace. Submit a pipeline using `submit`. When submit is called, a PipelineRun is created which in turn creates StepRun objects for each step in the workflow.

In [32]:
# build pipeline & run experiment
pipeline = Pipeline(workspace, steps=[prep_step, train_step])
run = exp.submit(pipeline)

Created step prepare step [48cfbf80][506c8be5-48d7-48af-844a-dc90de8ae3b7], (This step will run and generate new outputs)
Created step train step [0cf4bb29][05fccfd1-52e9-40fe-a1c5-ed8715fdffd4], (This step will run and generate new outputs)
Submitted PipelineRun f329cb80-6975-47a5-915c-706f71b5f836
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/f329cb80-6975-47a5-915c-706f71b5f836?wsid=/subscriptions/b17f1c19-34a2-47b8-a207-40ea477828fc/resourcegroups/aml-resource-group/workspaces/aml-workspace&tid=0f823349-2c12-431b-a03c-b2c0a43d6fb4


### Monitor the PipelineRun

In [33]:
run.wait_for_completion(show_output=True)

PipelineRunId: f329cb80-6975-47a5-915c-706f71b5f836
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/f329cb80-6975-47a5-915c-706f71b5f836?wsid=/subscriptions/b17f1c19-34a2-47b8-a207-40ea477828fc/resourcegroups/aml-resource-group/workspaces/aml-workspace&tid=0f823349-2c12-431b-a03c-b2c0a43d6fb4
PipelineRun Status: Running


StepRunId: cab3f93e-ecb7-4206-a1ce-3c7f7d48ca84
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/cab3f93e-ecb7-4206-a1ce-3c7f7d48ca84?wsid=/subscriptions/b17f1c19-34a2-47b8-a207-40ea477828fc/resourcegroups/aml-resource-group/workspaces/aml-workspace&tid=0f823349-2c12-431b-a03c-b2c0a43d6fb4
StepRun( prepare step ) Status: Running

Streaming azureml-logs/20_image_build_log.txt
2022/02/23 21:03:45 Downloading source code...
2022/02/23 21:03:46 Finished downloading source code
2022/02/23 21:03:46 Creating Docker network: acb_default_network, driver: 'bridge'
2022/02/23 21:03:47 Successfully set up Docker network: acb_default_network
2022/0



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base -c defaults conda


#
# To activate this environment, use
#
#     $ conda activate /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad
#
# To deactivate an active environment, use
#
#     $ conda deactivate

Removing intermediate container 901dde8ac809
 ---> 368b426e2430
Step 9/21 : ENV PATH /azureml-envs/azureml_da3e97fcb51801118b8e80207f3e01ad/bin:$PATH
 ---> Running in e17b259eb6c1
Removing intermediate container e17b259eb6c1
 ---> cf9499b1b0e2
Step 10/21 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> 5c3bc3563311
Step 11/21 : RUN echo "Copying environment context"
 ---> Running in 36efee671735
Copying environment context
Removing intermediate container 36efee671735
 ---> 57aefc0af72e
Step 12/21 : COPY azureml-environment-setup/environment_context.js


StepRun(prepare step) Execution Summary
StepRun( prepare step ) Status: Finished
{'runId': 'cab3f93e-ecb7-4206-a1ce-3c7f7d48ca84', 'target': 'gpu-cluster', 'status': 'Completed', 'startTimeUtc': '2022-02-23T21:15:26.433183Z', 'endTimeUtc': '2022-02-23T21:16:54.085128Z', 'services': {}, 'properties': {'ContentSnapshotId': 'f53d23bb-04fd-4725-b0f3-1f3ae8dcaf90', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.moduleid': '506c8be5-48d7-48af-844a-dc90de8ae3b7', 'azureml.moduleName': 'prepare step', 'azureml.runsource': 'azureml.StepRun', 'azureml.nodeid': '48cfbf80', 'azureml.pipelinerunid': 'f329cb80-6975-47a5-915c-706f71b5f836', 'azureml.pipeline': 'f329cb80-6975-47a5-915c-706f71b5f836', 'azureml.pipelineComponent': 'masterescloud', '_azureml.ComputeTargetType': 'amlcompute', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [{'dataset': {'id': '1e75dbf8-eeb6-4b01-b461-09dda4d94f32'

ad877dc53455: Verifying Checksum
ad877dc53455: Download complete
82770c3b2808: Verifying Checksum
82770c3b2808: Download complete
c991e53ceb53: Verifying Checksum
c991e53ceb53: Download complete
afa783568628: Verifying Checksum
afa783568628: Download complete
2b2d92136c10: Pull complete
0b412d2669ed: Pull complete
805003cfcee6: Pull complete
ad877dc53455: Pull complete
afa783568628: Pull complete
d67cf86adb17: Pull complete
82770c3b2808: Pull complete
c991e53ceb53: Pull complete
Digest: sha256:024c1f016bc4fe902601239d41f526ea987816ba25b524c22c4cb3cdd8db6ebf
Status: Downloaded newer image for mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20220113.v1@sha256:024c1f016bc4fe902601239d41f526ea987816ba25b524c22c4cb3cdd8db6ebf
 ---> 54296612fc48
Step 2/21 : USER root
 ---> Running in 27cb60add174
Removing intermediate container 27cb60add174
 ---> 3e55cb962061
Step 3/21 : RUN mkdir -p $HOME/.cache
 ---> Running in 990516ee4e96
Removing intermediate container 990516ee4e96
 ---> 03177c7678ae


Removing intermediate container 35f91ee6a2ba
 ---> 31bc18da98ad
Step 9/21 : ENV PATH /azureml-envs/azureml_f4d34986fac5bfa888344d2bd24de657/bin:$PATH
 ---> Running in 2e9a9c6db57e
Removing intermediate container 2e9a9c6db57e
 ---> 85c471d6b4d9
Step 10/21 : COPY azureml-environment-setup/send_conda_dependencies.py azureml-environment-setup/send_conda_dependencies.py
 ---> c3c4ee199ffd
Step 11/21 : RUN echo "Copying environment context"
 ---> Running in c10f33b88f5e
Copying environment context
Removing intermediate container c10f33b88f5e
 ---> 369aa5aa81a5
Step 12/21 : COPY azureml-environment-setup/environment_context.json azureml-environment-setup/environment_context.json
 ---> ca5caa7df44c
Step 13/21 : RUN python /azureml-environment-setup/send_conda_dependencies.py -p /azureml-envs/azureml_f4d34986fac5bfa888344d2bd24de657
 ---> Running in e02c1fdf8912
Report materialized dependencies for the environment
Reading environment context
Exporting conda environment
Sending request with mat


StepRun(train step) Execution Summary
StepRun( train step ) Status: Finished
{'runId': '5f49e800-63c2-4258-b41a-b1fa7eca5b52', 'target': 'gpu-cluster', 'status': 'Completed', 'startTimeUtc': '2022-02-23T21:22:02.01712Z', 'endTimeUtc': '2022-02-23T21:25:14.094486Z', 'services': {}, 'properties': {'ContentSnapshotId': 'f53d23bb-04fd-4725-b0f3-1f3ae8dcaf90', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.moduleid': '05fccfd1-52e9-40fe-a1c5-ed8715fdffd4', 'azureml.moduleName': 'train step', 'azureml.runsource': 'azureml.StepRun', 'azureml.nodeid': '0cf4bb29', 'azureml.pipelinerunid': 'f329cb80-6975-47a5-915c-706f71b5f836', 'azureml.pipeline': 'f329cb80-6975-47a5-915c-706f71b5f836', 'azureml.pipelineComponent': 'masterescloud', '_azureml.ComputeTargetType': 'amlcompute', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [{'dataset': {'id': '56114f5f-e138-4efb-842a-850cc9ab7086'}, 'con

'Finished'

In [38]:
run.find_step_run("train step")[0].get_metrics()

{'Loss': [0.8906879425048828,
  0.5625571608543396,
  0.492699533700943,
  0.44690972566604614,
  0.4149888753890991,
  0.3932421803474426,
  0.3724280893802643,
  0.3515191376209259,
  0.33642470836639404,
  0.33063381910324097],
 'Accuracy': [0.6644274592399597,
  0.7859039902687073,
  0.8146941065788269,
  0.8341107368469238,
  0.8454482555389404,
  0.8537282347679138,
  0.8626554012298584,
  0.8705336451530457,
  0.875756025314331,
  0.879527747631073],
 'Final test loss': 0.27688536047935486,
 'Final test accuracy': 0.9011366367340088,
 'Loss v.s. Accuracy': 'aml://artifactId/ExperimentRun/dcid.5f49e800-63c2-4258-b41a-b1fa7eca5b52/Loss v.s. Accuracy_1645651505.png'}

## Register the input dataset and the output model

Azure Machine Learning dataset makes it easy to trace how your data is used in ML.
For each Machine Learning experiment, you can easily trace the datasets used as the input through `Run` object.

In [45]:
# get input datasets
prep_step = run.find_step_run("prepare step")[0]
inputs = prep_step.get_details()["inputDatasets"]
input_dataset = inputs[0]["dataset"]

# list the files referenced by input_dataset
input_dataset.to_path()

['/mnist-fashion/t10k-images-idx3-ubyte',
 '/mnist-fashion/t10k-labels-idx1-ubyte',
 '/mnist-fashion/train-images-idx3-ubyte',
 '/mnist-fashion/train-labels-idx1-ubyte']

Register the input Fashion MNIST dataset with the workspace so that you can reuse it in other experiments or share it with your colleagues who have access to your workspace.

In [48]:
fashion_ds = input_dataset.register(
    workspace=workspace,
    name="fashion_ds",
    description="image and label files from fashion mnist",
    create_new_version=True,
)
fashion_ds

{
  "source": [
    "https://data4mldemo6150520719.blob.core.windows.net/demo/mnist-fashion"
  ],
  "definition": [
    "GetFiles"
  ],
  "registration": {
    "id": "812f759f-8813-4b5a-8e34-d1548620a2b2",
    "name": "fashion_ds",
    "version": 1,
    "description": "image and label files from fashion mnist",
    "workspace": "Workspace.create(name='aml-workspace', subscription_id='b17f1c19-34a2-47b8-a207-40ea477828fc', resource_group='aml-resource-group')"
  }
}

Register the output model with dataset

In [ ]:
run.find_step_run("train step")[0].register_model(
    model_name="keras-model",
    model_path="outputs/model/",
    datasets=[("train test data", fashion_ds)],
)